# Seattle Building Data Pre-processing
## Team 5 - Connor, John, Libby, & Natalie

This file contains the code used to preprocess and clean our selected dataset. The final step will be outputting a cleaned and processed dataset as a new .csv file.

#### Setup

In [1]:
#Makes paths work if you just clone or pull the repo
import os
os.chdir('../')


In [2]:
#Install Required Libraries (Only needs to run once)
%pip install -q pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
#import libraries
import pandas as pd

#### Import Data

In [4]:
#Paths
PATH_TO_DATASET = "Data/Raw Data/2022_Building_Energy_Benchmarking_20240906.csv"

In [5]:
#Import Dataset
building_DF = pd.read_csv(PATH_TO_DATASET) #raw data

print(building_DF.columns)

Index(['OSEBuildingID', 'DataYear', 'BuildingName', 'BuildingType',
       'TaxParcelIdentificationNumber', 'Address', 'City', 'State', 'ZipCode',
       'Latitude', 'Longitude', 'Neighborhood', 'CouncilDistrictCode',
       'YearBuilt', 'NumberofFloors', 'NumberofBuildings', 'PropertyGFATotal',
       'PropertyGFABuilding(s)', 'PropertyGFAParking', 'ENERGYSTARScore',
       'SiteEUIWN(kBtu/sf)', 'SiteEUI(kBtu/sf)', 'SiteEnergyUse(kBtu)',
       'SiteEnergyUseWN(kBtu)', 'SourceEUIWN(kBtu/sf)', 'SourceEUI(kBtu/sf)',
       'EPAPropertyType', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType',
       'ThirdLargestPropertyUseTypeGFA', 'Electricity(kWh)', 'SteamUse(kBtu)',
       'NaturalGas(therms)', 'ComplianceStatus', 'ComplianceIssue',
       'Electricity(kBtu)', 'NaturalGas(kBtu)', 'TotalGHGEmissions',
       'GHGEmissionsIntensity'],
      dtype='object')


#### Data Cleaning

In [6]:
#Drop Columns That We Decided Not To Use
Dropped_Columns = ['TaxParcelIdentificationNumber', 'City', 'State', 'CouncilDistrictCode', 'PropertyGFABuilding(s)', 
                   'PropertyGFAParking', 'SiteEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 
                   'SourceEUIWN(kBtu/sf)', 'LargestPropertyUseType', 'LargestPropertyUseTypeGFA', 
                   'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType',
                   'ThirdLargestPropertyUseTypeGFA', 'Electricity(kWh)', 'NaturalGas(therms)', 'TotalGHGEmissions']
#Drop Listed Columns
df_after_drop = building_DF.drop(columns=Dropped_Columns)

In [7]:
# Column wise Null counts
column_nulls = df_after_drop.isnull().sum()
sorted_column_nulls = column_nulls[column_nulls > 0].sort_values(ascending=False)
column_dtypes = df_after_drop.dtypes
nulls_and_dtypes = pd.DataFrame({
    'Null Count': sorted_column_nulls,
    'Data Type': column_dtypes[sorted_column_nulls.index]
})

print(nulls_and_dtypes)

                       Null Count Data Type
SteamUse(kBtu)               3517   float64
NaturalGas(kBtu)             1669   float64
ENERGYSTARScore              1174   float64
SourceEUI(kBtu/sf)            458   float64
SiteEUI(kBtu/sf)              458   float64
EPAPropertyType               234    object
GHGEmissionsIntensity         209   float64
Electricity(kBtu)             208   float64
Neighborhood                    1    object


In [8]:
def handle_null_values(df):
    '''This function takes the building dataframe and returns a new dataframe with the null values handled'''
    df['SteamUse(kBtu)'] = df['SteamUse(kBtu)'].fillna(0.0) #Buildings that don't have steam have zero steam use
    df['NaturalGas(kBtu)'] = df['NaturalGas(kBtu)'].fillna(0.0) #Buildings that don't have natural gas use zero natural gas
    df['ENERGYSTARScore'] = df['ENERGYSTARScore'] #not sure what to do with this yet
    df['SourceEUI(kBtu/sf)'] = df['SourceEUI(kBtu/sf)'].fillna(df['SourceEUI(kBtu/sf)'].mean()) #Fill with mean
    df['SiteEUI(kBtu/sf)'] = df['SiteEUI(kBtu/sf)'].fillna(df['SiteEUI(kBtu/sf)'].mean()) #Fill with mean
    df['EPAPropertyType'] = df['EPAPropertyType'].fillna('Other') #Fill with 'Other'
    df['GHGEmissionsIntensity'] = df['GHGEmissionsIntensity'].fillna(df['GHGEmissionsIntensity'].mean()) #drop Rows with Nulls
    df['Electricity(kBtu)'] = df['Electricity(kBtu)'].fillna(df['Electricity(kBtu)'].mean()) #leaving as zero electricity use for now but needs to be fixed
    df.dropna(subset=['Neighborhood']) #drop the single row with a null, goodbye :(
        
    return df

df_nulls_cleaned = handle_null_values(df_after_drop)

df_nulls_cleaned.to_csv('Data/Seattle_Building_Data_Cleaned.csv', index=False)

#### Processing Steps

TODO: Change epa property types from 'other - random stuff' to just 'other'
onehot encode categorical columns
normalize numerical columns

In [9]:
cleandf = pd.read_csv('Data\Seattle_Building_Data_Cleaned.csv')

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\jfsal\AppData\Local\Temp\ipykernel_8316\1551968840.py:1: SyntaxWarning: invalid escape sequence '\S'
  cleandf = pd.read_csv('Data\Seattle_Building_Data_Cleaned.csv')


In [10]:
#EPA Property Type Cleaning 
for index, row in cleandf.iterrows():
    if 'other' in row['EPAPropertyType'].lower():
        cleandf.at[index, 'EPAPropertyType'] = 'Other'
        
COMBINE_LOW_COUNTS = True #Change to false if we dont want to do this step
if COMBINE_LOW_COUNTS:
     value_counts = cleandf['EPAPropertyType'].value_counts()
     cleandf['EPAPropertyType'] = cleandf['EPAPropertyType'].apply(lambda x: 'Other' if value_counts[x] < 10 else x)
     
print(cleandf['EPAPropertyType'].value_counts())

EPAPropertyType
Multifamily LR (1-4)                     1004
Multifamily MR (5-9)                      736
Office                                    468
Other                                     434
K-12 School                               148
Non-Refrigerated Warehouse                138
Mixed Use Property                        124
Multifamily HR (10+)                      124
Hotel                                      74
Retail Store                               67
Worship Facility                           55
Medical Office                             45
Self-Storage Facility                      37
Distribution Center                        37
Supermarket/Grocery Store                  35
College/University                         26
Laboratory                                 20
Parking                                    20
Manufacturing/Industrial Plant             16
Refrigerated Warehouse                     13
Hospital (General Medical & Surgical)      11
Residence Hall/Dor

In [11]:
print(cleandf.columns)

Index(['OSEBuildingID', 'DataYear', 'BuildingName', 'BuildingType', 'Address',
       'ZipCode', 'Latitude', 'Longitude', 'Neighborhood', 'YearBuilt',
       'NumberofFloors', 'NumberofBuildings', 'PropertyGFATotal',
       'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SourceEUI(kBtu/sf)',
       'EPAPropertyType', 'SteamUse(kBtu)', 'ComplianceStatus',
       'ComplianceIssue', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
       'GHGEmissionsIntensity'],
      dtype='object')


In [17]:
#encoding
from sklearn.preprocessing import StandardScaler, OneHotEncoder

cat_cols = ['Neighborhood', 'EPAPropertyType', 'ComplianceStatus', 'ComplianceIssue', 'BuildingType']
num_cols = ['SiteEUI(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SteamUse(kBtu)', 'NaturalGas(kBtu)', 'Electricity(kBtu)']

#One Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
encoded_cols = encoder.fit_transform(cleandf[cat_cols])
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(cat_cols))

# Standard Scaling
scaler = StandardScaler()
scaled_cols = scaler.fit_transform(cleandf[num_cols])
scaled_df = pd.DataFrame(scaled_cols, columns=num_cols)

# Remove original Columns
processed_df = pd.concat([cleandf.reset_index(drop=True), encoded_df, scaled_df], axis=1)
processed_df = processed_df.drop(columns=cat_cols + num_cols)
print(processed_df.head())



   OSEBuildingID  DataYear                    BuildingName         Address  \
0              1      2022            MAYFLOWER PARK HOTEL   405 OLIVE WAY   
1              2      2022                 PARAMOUNT HOTEL     724 PINE ST   
2              3      2022  WESTIN HOTEL (Parent Building)    1900 5TH AVE   
3              5      2022                       HOTEL MAX  620 STEWART ST   
4              8      2022           WARWICK SEATTLE HOTEL   401 LENORA ST   

   ZipCode  Latitude  Longitude  YearBuilt  NumberofFloors  NumberofBuildings  \
0    98101  47.61220 -122.33799       1927              12                  1   
1    98101  47.61307 -122.33361       1996              11                  1   
2    98101  47.61367 -122.33822       1969              41                  3   
3    98101  47.61412 -122.33664       1926              10                  1   
4    98121  47.61375 -122.34047       1980              18                  1   

   ...  \
0  ...   
1  ...   
2  ...   
3  .

#### Export Processed Dataset

In [18]:
processed_df.to_csv('Data/Building_Data_Preprocessed.csv', index=False)